In [1]:
import pandas as pd
import numpy as np
import pickle
from keras.models import load_model

from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import plot_precision_recall_curve
import matplotlib.pyplot as plt

# with open('../structure_data_remove_Lymphocyte_Troponin/LightGBM_withoutCXR_v2_AUC0.8103.pickle', 'rb') as f:
#     LightGBM = pickle.load(f)
# with open('../structure_data_remove_Lymphocyte_Troponin/CatBoost_AUC0.7812.pickle', 'rb') as f:
#     CatBoost = pickle.load(f)
# with open('../structure_data_remove_Lymphocyte_Troponin/XGB_AUC0.8457.pickle', 'rb') as f:
#     XGBoostClassifier = pickle.load(f)
# with open('../structure_data_remove_Lymphocyte_Troponin/RF_AUC0.8295.pickle', 'rb') as f:
#     RandomForest = pickle.load(f)
# xception = load_model('../CXR/xception_mse0.9084_val_mse1.1377.h5')

In [2]:
train_ID_score = pd.read_excel("../CXR/train_test_split.xlsx", sheet_name = 'train')
test_ID_score = pd.read_excel("../CXR/train_test_split.xlsx", sheet_name = 'test')

train_ID_list = [int(i.split('_')[0]) for i in list(train_ID_score.ID)]
test_ID_list = [int(i.split('_')[0]) for i in list(test_ID_score.ID)]

FileNotFoundError: [Errno 2] No such file or directory: '../CXR/train_test_split.xlsx'

In [ ]:
### NTUH dataset
dataset = pd.read_csv('../concat_influ8_severity_score.csv')
dataset = dataset[['ID', 'Gender', 'Age', 'BMI', 'Diastolic_blood_pressure', 'Heart_rate', 
                   'Respiratory_rate', 'Creatinine', 'CRP', 'Hemoglobin', 'Bicarbonate', 
                   'PCO2', 'pH', 'Platelet', 'PO2',  'Blood_urea_nitrogen', 
                   'Lactic_acid', 'INR', 'Glucose', 'Hematocrit', 'severity_score', 'Survival_30']]
train = dataset.loc[dataset['ID'].isin(train_ID_list)]  #534筆(佔69.8%)
train = train.drop('ID', axis = 1)
test = dataset.loc[dataset['ID'].isin(test_ID_list)]  #231筆(佔30.2%)
test = test.drop('ID', axis = 1)

### random sampling do not have good results
# X_train = train.drop(['severity_score', 'Survival_30'], axis = True)
# X_test = test.drop(['severity_score', 'Survival_30'], axis = True)
# y_train = train[['Survival_30']]
# y_test = test[['Survival_30']]

### Resampling
count_class_0, count_class_1 = train.Survival_30.value_counts()
df_class_0 = train[train['Survival_30'] == 0]
df_class_1 = train[train['Survival_30'] == 1]
df_class_1_over = df_class_1.sample(count_class_0, replace=True)
df_test_over = pd.concat([df_class_0, df_class_1_over], axis=0)

### validation ###
X_train = df_test_over.drop(['severity_score', 'Survival_30'], axis = 1)
y_train = df_test_over[['Survival_30']]
X_test = test.drop(['severity_score', 'Survival_30'], axis = 1)
y_test = test[['Survival_30']]

In [ ]:
### Italian dataset
external_dataset = pd.read_csv("../concat_italy_severity_score.csv")
# ##
# external_dataset = external_dataset[external_dataset.isnull().sum(axis=1) <= 5]
# ##
external_dataset = external_dataset[['Gender', 'Age', 'BMI', 'DBP (mmHg)', 'Heart rate ', 'RR', 
                                  'Creatinine ', 'CRP', 'Hb', 'HCO3-', 'pCo2', 'Ph',
                                  'PLT', 'pO2', 'BUN ', 'Lactate', 'INR ', 'Glycemia', 
                                  'HCT', 'severity_score','30 days mortality (0 alive, 1 death)']]
external_dataset.columns = ['Gender', 'Age', 'BMI', 'Diastolic_blood_pressure', 'Heart_rate', 
                   'Respiratory_rate', 'Creatinine', 'CRP', 'Hemoglobin', 'Bicarbonate', 
                   'PCO2', 'pH', 'Platelet', 'PO2',  'Blood_urea_nitrogen', 
                   'Lactic_acid', 'INR', 'Glucose', 'Hematocrit', 'severity_score', 'Survival_30']
external_dataset['Gender'] = external_dataset['Gender'].replace('F',0).replace('M',1)
external_dataset['BMI'] = external_dataset['BMI'].replace("Normal", "18.5")
external_dataset['BMI'] = external_dataset['BMI'].replace("Overweight", "21.75")
external_dataset['BMI'] = external_dataset['BMI'].replace("Obese ", "27.5")
external_dataset['BMI'] = external_dataset['BMI'].replace("Underweight", "30")
external_dataset['Gender'] = pd.to_numeric(external_dataset['Gender'], errors='coerce')
external_dataset['BMI'] = pd.to_numeric(external_dataset['BMI'], errors='coerce')

X_external = external_dataset.drop(['severity_score', 'Survival_30'], axis = 1)
y_external = external_dataset[['Survival_30']]

In [ ]:
external_dataset = pd.read_csv("../concat_italy_severity_score.csv")
# ##
# external_dataset = external_dataset[external_dataset.isnull().sum(axis=1) <= 5]
# ##
external_dataset = external_dataset[['Gender', 'Age', 'BMI', 'DBP (mmHg)', 'Heart rate ', 'RR', 
                                  'Creatinine ', 'CRP', 'Hb', 'HCO3-', 'pCo2', 'Ph',
                                  'PLT', 'pO2', 'BUN ', 'Lactate', 'INR ', 'Glycemia', 
                                  'HCT', 'severity_score','30 days mortality (0 alive, 1 death)']]
external_dataset['Gender'] = external_dataset['Gender'].replace('F',0).replace('M',1)
external_dataset['BMI'] = external_dataset['BMI'].replace("Normal", "18.5")
external_dataset['BMI'] = external_dataset['BMI'].replace("Overweight", "21.75")
external_dataset['BMI'] = external_dataset['BMI'].replace("Obese ", "27.5")
external_dataset['BMI'] = external_dataset['BMI'].replace("Underweight", "30")
external_dataset['Gender'] = pd.to_numeric(external_dataset['Gender'], errors='coerce')
external_dataset['BMI'] = pd.to_numeric(external_dataset['BMI'], errors='coerce')
external_dataset = external_dataset.fillna(round(external_dataset.mean(), 2))

external_dataset.columns = ['Gender', 'Age', 'BMI', 'Diastolic_blood_pressure', 'Heart_rate', 
                   'Respiratory_rate', 'Creatinine', 'CRP', 'Hemoglobin', 'Bicarbonate', 
                   'PCO2', 'pH', 'Platelet', 'PO2',  'Blood_urea_nitrogen', 
                   'Lactic_acid', 'INR', 'Glucose', 'Hematocrit', 'severity_score', 'Survival_30']

X_external_RF = external_dataset.drop(['severity_score', 'Survival_30'], axis = 1)
y_external_RF = external_dataset[['Survival_30']]

In [6]:
# train
xception_pred_train = df_test_over[['severity_score']]
RandomForest_predict_proba_train = RandomForest.predict_proba(X_train)[:, 1]


# test
xception_pred_test = test[['severity_score']]
RandomForest_predict_proba_test = RandomForest.predict_proba(X_test)[:, 1]

# external
xception_pred_external = external_dataset[['severity_score']]
RandomForest_predict_proba_external = RandomForest.predict_proba(X_external_RF)[:, 1]

In [7]:
xception_pred_train["RandomForest"] = RandomForest_predict_proba_train
xception_pred_test["RandomForest"] = RandomForest_predict_proba_test
xception_pred_external["RandomForest"] = RandomForest_predict_proba_external

D:\Users\122156\AppData\Local\Temp/ipykernel_20136/2565043229.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xception_pred_train["LightGBM"] = LightGBM_predict_proba_train
D:\Users\122156\AppData\Local\Temp/ipykernel_20136/2565043229.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xception_pred_test["LightGBM"] = LightGBM_predict_proba_test
D:\Users\122156\AppData\Local\Temp/ipykernel_20136/2565043229.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.


# best model

In [ ]:
# Random Forest
model_file_name='./Late_Fusion_RF_lr_AUC0.8371.pickle'
with open(model_file_name, 'rb') as f:
    model = pickle.load(f)

In [ ]:
from sklearn import preprocessing, linear_model
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.calibration import calibration_curve
from sklearn.metrics import classification_report
import seaborn as sns

X_train = xception_pred_train[["severity_score", "RandomForest"]]
X_test = xception_pred_test[["severity_score", "RandomForest"]]
X_external = xception_pred_external[["severity_score", "RandomForest"]]

y_train = train[['Survival_30']]
y_test = test[['Survival_30']]
y_external = external_dataset[['Survival_30']]

y_prob = model.predict_proba(X_test)
y_pred = model.predict(X_test)
y_prob_ex = model.predict_proba(X_external)
y_pred_ex = model.predict(X_external)

### validation ###
cm = metrics.confusion_matrix([x for [x] in y_test.values.tolist()], y_pred)
tn, fp, fn, tp = cm.ravel()
sensitivity = tp / (tp+fn+1e-8)
specificity = tn / (tn+fp+1e-8)
precision = tp/(tp+fp+1e-8)
recall = tp/(tp+fn+1e-8)
f1 = (2*precision*recall)/(precision+recall+1e-8)

print("validation")
print('AUC: {:.4f}'.format(metrics.roc_auc_score(y_test, y_prob[:, 1])))
print('Acc: {:.4f}'.format(metrics.accuracy_score(y_test, y_pred)))
print('sensitivity: {:.4f}'.format(sensitivity))
print('specificity: {:.4f}'.format(specificity))
print('precision: {:.4f}'.format(precision))
print('recall: {:.4f}'.format(recall))
print('F1: {:.4f}'.format(f1))

fpr, tpr, thresholds = metrics.roc_curve(y_test, y_prob[:,1])
AUC_PR = metrics.auc(tpr, fpr)
print('The area under the recall precision curve: {:.4f}'.format(AUC_PR))

print(classification_report(y_test.values.tolist(), y_pred, labels=[0,1]))

### Confusion Matrics
sns.set(font_scale = 1.4) # for label size
sns.heatmap(cm, annot=True, annot_kws={"size": 16},fmt='g') # font size
plt.show()

### Recall Precision curves
disp = plot_precision_recall_curve(model, X_test, y_test)
disp.ax_.set_title('2-class Precision-Recall curve')
disp.ax_.legend(loc = 'upper right')
plt.show()

### ROC
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc_score(y_test, y_prob[:, 1]))
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

### external ###
cm = metrics.confusion_matrix([x for [x] in y_external.values.tolist()], y_pred_ex)
tn, fp, fn, tp = cm.ravel()
sensitivity = tp / (tp+fn+1e-8)
specificity = tn / (tn+fp+1e-8)
precision = tp/(tp+fp+1e-8)
recall = tp/(tp+fn+1e-8)
f1 = (2*precision*recall)/(precision+recall+1e-8)

print("external test")
print('AUC: {:.4f}'.format(metrics.roc_auc_score(y_external, y_prob_ex[:, 1])))
print('Acc: {:.4f}'.format(metrics.accuracy_score(y_external, y_pred_ex)))
print('sensitivity: {:.4f}'.format(sensitivity))
print('specificity: {:.4f}'.format(specificity))
print('precision: {:.4f}'.format(precision))
print('recall: {:.4f}'.format(recall))
print('F1: {:.4f}'.format(f1))

fpr, tpr, thresholds = metrics.roc_curve(y_external, y_prob_ex[:,1])
AUC_PR = metrics.auc(tpr, fpr)
print('The area under the recall precision curve: {:.4f}'.format(AUC_PR))

print(classification_report(y_external.values.tolist(), y_pred_ex, labels=[0,1]))

### Confusion Matrics
sns.set(font_scale = 1.4) # for label size
sns.heatmap(cm, annot=True, annot_kws={"size": 16},fmt='g') # font size
plt.show()

### Recall Precision curves
disp = plot_precision_recall_curve(model, X_external, y_external)
disp.ax_.set_title('2-class Precision-Recall curve')
disp.ax_.legend(loc = 'upper right')
plt.show()

### ROC
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc_score(y_external, y_prob_ex[:, 1]))
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()